In this section, we cover the basics of azure ML. We will propose how to gradually move an existing ML project to Azure ML. This will give the same benefits as starting a completely new project in azure ML, ranging from model tracking to automated deployments and auto-scaling training clusters. The following topics will be covered:
* Demystifying the different azure services for ML
* Azure cognitive services and custom vision
* Azure ML tools with GUIs
* Azure ML service

# Demystifying the different azure services for ML
Azure offers many services that can be used to perform Ml -you can use a simple virtual machine, a pre-configured VM for ML(Data Science Virtual Machine), Azure Notebooks using a shared free kernel, or any other service that gives you compute resources and data storage. <br>
First, it is important to discuss the difference between a simple compute resource, an ML infrastructure servce, and an ML modeling service.
* A **compute resource:** can be any service in Azure that provides you with computing power, such as VMs, managed cluster of VMs(Azure Batch, Azure Databricks etc), container execution engines(Azure Kubernetes services etc), or hybrid compute services such as app service
* An **ML infrastructure service** helps you implement, orchestrate, automate and optimize your ML training, pipelines and deployments. USing such a service, you would usually implement your own preprocessing and ML algorithms using your own frameworks. However, the service would support you with infrastructure for the training, optimization and deployment process.
* Finally, an **ML modeling service** is a service that helps you create or use ML models without writing your own code. Services such as Cogitive services, Azure Automated ML(AutoML), Azure ML studio, and custom vision.

The following pipelines is a typical choice of Azure Ml and compute services in a real-world example for a predictive quality model that can be deployed on an on-premise edge device for stream processing:
* Track experiments, pipelines, trained models and containers using Azure Ml
* Run experiments in azure notebook using a DSVM
* Label the dataset using custom vision
* Create an object detection mode without code using custom vision
* Deploy the model to azure ioT Edge

## Choosing an Azure service for ML
The trade-offs are similar when evaluating a managed **Platform-as-a-Service (Paas)** offering versus the more flexible **Infrastructure-as-a-Service (IaaS)** software. In general, the Azure ML services can be ordered by increasing flexibility and operational effort. Here is a list in sorted order of its flexibility and operational effort:
* A fully managed service with pre-trained models for scoring only
* A managed service with pre-trained models and customization through transfer learning
* A managed service with GUI to experiment, build, train and deploy models
* A managed service to orchestrate compute resources and facilitate ML processes
* Individual compute services to experiment, train and deploy models


The following is a guideline towards choosing the correct ML service according to the logic that we've just discussed:<br>
h

![](/home/kevin/Desktop/cwd/azure/azure-machine-learning/images/01-guideline.png)

The diagram can be explained in the following way:
1. Can you consume a pretrained model or do you have to build a custom model on your own? If the model already exists, you do not have to build one of your own.
2. If you are building on top of an existing algorithm but for a specialized domain, you can use custom cognitive services to fine-tune a pre-trained model for your own domain. If you want to detect faulty products on a production line using camera images, you can use custom vision for fine-tuning teh vision cognitive service API for the domain of your products. <br>
In azure, you have various choices to build your end-to-end MLpipelines for training, optimizing, and deploying custom ML models:
* Build your own tools
* Use open source tools such as Azure Databricks with ML flow
* USing a GUI tool, such as azure ml studio
* USing azure ML

There are 2 diffeerent compute targets which both runa  python interpreter:
* **The authoring runtime:** is a lightweight python runtime used to call the azure ML SDK and orchestrate your infrastructure. You usually use it to create compute rsources, trigger training runs, build environments, and deploy models. No real compute is done in this environment, and hence lightweight python interpreters such as vs code running python locally, an azure notebook, or a single Azure ML compute instance are commonly used.
* **The execution environment:** is used to execute your ML training or scoring code. The python interpreter does the real work, you likely need a beefy machine if you need to train a deep learning model. You will often use Azure ML training cluster(AML compute), Azure Kubernetes servie(AKS), or Azure container instance(ACI) as execution environments.

In a typical workflow, you can use an authoring workflow environment to create a compute cluster (the execution environment) through the Azure ML SDK, and then submit the training script to this cluster again using the Azure ML SDK. This is how:


In [9]:
 from azureml.core.authentication import InteractiveLoginAuthentication
 ia = InteractiveLoginAuthentication(tenant_id='292035c0-a3ff-49d4-9b39-8cab5c9e0b53')
 # You can find tenant id under azure active directory->properties
 ws = Workspace.get(name='main-workspace',
                    subscription_id="167b9ae9-f575-4021-810b-253a2e5be2bd",
                     resource_group='MLOps',auth=ia)

exp = Experiment(
    workspace = ws,
    name = "cifar10_cnn_experiment"
)

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Create the cluster
config = AmlCompute.provisioning_configuration(
    vm_size = 'STANDARD_D2_V2',
    max_nodes = 4
)
cluster = ComputeTarget.create(ws, 'cluster', config)
cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
from azureml.train.estimator import Estimator

# submit the training script to the cluster
estimator = Estimator(
    compute_target = cluster,
    source_directory = './code',
    entry_script = 'train.py',
    conda_packages = ['keras', 'tensorflow']
)
run = exp.submit(estimator)
run.wait_for_completion(show_output=True)

RunId: cifar10_cnn_experiment_1657623614_c5654425
Web View: https://ml.azure.com/runs/cifar10_cnn_experiment_1657623614_c5654425?wsid=/subscriptions/167b9ae9-f575-4021-810b-253a2e5be2bd/resourcegroups/MLOps/workspaces/main-workspace&tid=292035c0-a3ff-49d4-9b39-8cab5c9e0b53

Streaming azureml-logs/20_image_build_log.txt

2022/07/12 11:00:20 Downloading source code...
2022/07/12 11:00:21 Finished downloading source code
2022/07/12 11:00:21 Creating Docker network: acb_default_network, driver: 'bridge'
2022/07/12 11:00:22 Successfully set up Docker network: acb_default_network
2022/07/12 11:00:22 Setting up Docker configuration...
2022/07/12 11:00:22 Successfully set up Docker configuration
2022/07/12 11:00:22 Logging in to registry: bf3e5ed3854e46519ae484c182dde994.azurecr.io
2022/07/12 11:00:23 Successfully logged into bf3e5ed3854e46519ae484c182dde994.azurecr.io
2022/07/12 11:00:23 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2

{'runId': 'cifar10_cnn_experiment_1657623614_c5654425',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-07-12T11:13:26.841827Z',
 'endTimeUtc': '2022-07-12T11:14:15.561539Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '8c43f81b-5688-46b0-a15e-755e25682a0e',
  'azureml.git.repository_uri': 'https://github.com/Kevin-Nduati/azure-machine-learning.git',
  'mlflow.source.git.repoURL': 'https://github.com/Kevin-Nduati/azure-machine-learning.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'f834542784842be534c6e7500b76da5713d07abe',
  'mlflow.source.git.commit': 'f834542784842be534c6e7500b76da5713d07abe',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolut

NameError: name 'aa' is not defined